In [1]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

In [2]:
backGround = None
accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [3]:
def calc_accum_avg(frame,accumulated_weight):
    
    global backGround
    
    if backGround is None:
        backGround = frame.copy().astype('float')
        return None
    
    cv2.accumulated_weight(frame,backGround,accumulated_weight)

In [4]:
def segment(frame,threshold_min=25):
    
    diff = cv2.absdiff(backGround.astype('uint8'),frame)
    
    ret, thresh = cv2.threshold(diff,threshold_min,255,
                               cv2.THRESH_BINARY)
    
    image, cts, hierarchy = cv2.findContours(thresh.copy(),
                                            cv2.RETR_EXTERNAL,
                                            cv2.CHAIN_APPROX_SIMPLE)
    
    if len(cts) == 0:
        return None
    
    else:
        # The largest external contour in roi, is the hand.
        hand_segment = max(cts,key=cv2.contourArea)
        
        return (thresh,hand_segment)

In [5]:
def count_fingers(thresh,hand_segment):
    
    conv_hull = cv2.convexHull(hand_segment)
    
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()[0]])
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()[0]])
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()[0]])
    right = tuple(conv_hull[conv_hull[:,:,0].argmax()[0]])
    
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    distance = pairwise.euclidean_distances([cX,cY], Y=[left,right,top,bottom])[0]
    
    max_distance = distance.max()
    
    #Depends on the size of the hand
    radius = int(0.9 * max_distance)
    circumference = (2 * np.pi * radius)
    
    circular_roi = np.zeros(thresh[:2],dtype='uint8')
    
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    
    circular_roi = cv2.bitwise_and(thresh,thresh,
                                  mask=circular_roi)
    
    image, cts, hierarchy = cv2.findContours(circular_roi.copy(),
                                            cv2.RETR_EXTERNAL,
                                            cv2.CHAIN_APPROX_SIMPLE)
    
    
    count = 0
    
    for cnt in cts:
        
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        # Limits 
        out_of_wrist = (cY + (cY*0.25)) > (y+h)
        limit_points = ((circumference*0.25) > cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count += 1
            
    return count
    
    